In [1]:
import json
import re
import os
import sys
import numpy as np
import pandas as pd
import nltk
from tqdm import tqdm
pd.options.display.float_format = '{:,}'.format

In [2]:
def find_book_id(name):
    result = []
    name_words = set(map(lambda word: word.lower(), nltk.word_tokenize(name)))
    with open("data/goodreads_books_fantasy_paranormal.json") as f:
        for line in tqdm(f):
            book = json.loads(line)
            if name_words.issubset(set(map(lambda word: word.lower(), nltk.word_tokenize(book["title"])))):
                result.append({
                    "title" : book["title"],
                    "id" : book["book_id"],
#                     book["title"] : book["book_id"],
                    "ratings_count" : book["ratings_count"],
                    "text_reviews_count" : book["text_reviews_count"]
                })
#                 print(json.dumps(book, indent=2))
    return result

In [3]:
# stone_id = sorted(sorcerers + philosophers, key=lambda point: int(point["text_reviews_count"]), reverse=True)[0]["id"]

In [4]:
stone_id = "3"

In [5]:
def get_reviews_from_book_id(book_id):
    reviews = []
    with open("data/goodreads_reviews_fantasy_paranormal.json") as f:
        for line in tqdm(f):
            review = json.loads(line)
            if review["book_id"] == book_id:
                reviews.append(review)
    return reviews

In [6]:
reviews = get_reviews_from_book_id(stone_id)

3424641it [00:51, 66577.44it/s]


In [7]:
# text = " ".join(review["review_text"] for review in reviews)
# tokenized = nltk.word_tokenize(text)
# stopwords = nltk.corpus.stopwords.words("english")

In [8]:
def frequent_ngrams(words, n):
#     ngrams = ngram_maker(map(lambda word: word.lower(), words))
    from nltk.util import ngrams
    ngrams = ngrams(words, n)
    filtered_ngrams = [
        ngram for ngram in ngrams
        if all(gram.lower() not in stopwords and gram.isalnum() for gram in ngram)
    ]
    return nltk.FreqDist(filtered_ngrams)

In [9]:
# fd_bigrams = frequent_ngrams(tokenized, 2)
# fd_bigrams.most_common(10)

In [10]:
# fd_trigrams = frequent_ngrams(tokenized, 3)
# fd_trigrams.most_common(10)

In [11]:
from transformers import pipeline, AutoTokenizer, AutoConfig

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-04-20 11:32:44.168776: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-20 11:32:44.168803: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [35]:
name_pegasus = "google/pegasus-xsum"
name_bart = "sshleifer/distilbart-cnn-12-6"
# config = AutoConfig.from_pretrained(name, vocab_size=200000)
tokenizer_pegasus = AutoTokenizer.from_pretrained(name_pegasus)
tokenizer_bart = AutoTokenizer.from_pretrained(name_bart)

summarizer_pegasus = pipeline("summarization", model=name_pegasus, tokenizer=tokenizer_pegasus)
summarizer_bart = pipeline("summarization", model=name_bart, tokenizer=tokenizer_bart)
## [{ "summary_text": " Paris is the capital and most populous city of France..." }]

In [13]:
filtered = list(filter(lambda review: 100 < len(review["review_text"].split()) < 120, reviews))
longest = max(filtered, key=lambda review: len(review["review_text"].split(" ")))
votest = max(filtered, key=lambda review: int(review["n_votes"]))

In [14]:
longest

{'user_id': 'f5d62d00679e9083961474707c688a26',
 'book_id': '3',
 'review_id': '3996e59e1972eb80efd33f7b8c2863ca',
 'rating': 5,
 'review_text': 'This review also appears on BookDigits, which offers a brand new way of rating, reviewing, and discovering books. \n Grade: A- \n L/C Ratio: 30% Literary / 70% Commercial \n Thematic Breakdown: \n 30% - Magic \n 25% - Adventure \n 20% - Friendship \n 15% - Mystery \n 10% - School \n Addictiveness: High \n Movie Potential: 2 Thumbs Up \n Re-readability: High \n This is my second time through the series. More than a decade has passed since my first reading of "The Sorcerer\'s Stone." Both Rowling\'s world-building and Harry\'s origin story remain as awesome as I remember them. The mysteries and revelations that dominate the second half of the book can be somewhat clumsy and convenient, but they succeed in raising the stakes of the plot.',
 'date_added': 'Mon Jun 10 06:07:39 -0700 2013',
 'date_updated': 'Mon Jun 24 09:19:06 -0700 2013',
 'read_

In [15]:
summarizer_bart(longest["review_text"])

[{'summary_text': " Rowling's world-building and Harry's origin story remain as awesome as I remember them . The mysteries and revelations that dominate the second half of the book can be somewhat clumsy and convenient, but they succeed in raising the stakes of the plot . This review also appears on BookDigits, which offers a brand new way of rating, reviewing, and discovering books ."}]

In [16]:
votest

{'user_id': '2202e3ba89415e8237022731037be834',
 'book_id': '3',
 'review_id': '3b814ddac8bb0e12d27f1056318c282c',
 'rating': 4,
 'review_text': 'My boyfriend and I are read this book together this summer. My goal was to get him to read more since it is one of my favorite hobbies, so I let him pick the book we read. He chose Harry Potter. Well, I guess it was about time I finally read this book :P Not to brag or anything... but finished before him ;) haha. I enjoyed this book. It was a really good kids book. It is a fast, easy read, but like I said earlier it was defiantly written for someone younger. \n I would give this book a 3.5 stars, so I am rounding it up to 4 stars!',
 'date_added': 'Thu Jul 25 21:15:35 -0700 2013',
 'date_updated': 'Thu Jul 14 20:34:36 -0700 2016',
 'read_at': 'Thu Jul 14 00:00:00 -0700 2016',
 'started_at': 'Sun Jun 26 00:00:00 -0700 2016',
 'n_votes': 26,
 'n_comments': 4}

In [17]:
summarizer_bart(votest["review_text"])

Your max_length is set to 142, but you input_length is only 134. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=67)


[{'summary_text': ' My boyfriend and I are read this book together this summer . It was a really good kids book . It is a fast, easy read, but like I said earlier it was defiantly written for someone younger . I would give this book a 3.5 stars, so I am rounding it up to 4 stars!'}]

In [19]:
real_longest = max(reviews, key=lambda review: len(review["review_text"].split(" ")))

In [20]:
real_longest["review_text"]

'** spoiler alert ** \n [review no longer makes sense, since I wrote it when I was sixteen. I\'m seventeen now. And less hard on the Dursleys.] \n I\'m sixteen years old, and until yesterday, I\'d never read a Harry Potter book. \n That\'s right, I\'m The Girl Who Lived. Without Harry Potter. (Heh, that was a joke.) \n I have always professed to be a compulsive reader, but no one really believes me after they find out I haven\'t read Harry Potter. It seems to them like kind of a huge thing to have missed. I can understand that, but if, like me, you\'ve never read this series, trust me--I am you. I feel your pain. \n So yesterday I finally read the first book, after years of sheepishly admitting to others that I hadn\'t read my generation\'s most popular series, after years of learning all about Harry and Ron and Voldemort and Hermione secondhand. \n Enough preliminaries. It\'s time for literary comparisons. \n J.K. Rowling is a really excellent storyteller. Her voice reminded me of man

In [49]:
def oversummarize(summarizer, text, rounds=3, step=1000, verbosity=0):
    for r in range(1, rounds + 1):
        if verbosity > 0:
            print(f"ROUND {r}")
        
        summary = []
        for i in range(0, len(text), step):
            step_text = text[i:i + step]
            step_summary = summarizer(step_text)[0]["summary_text"]
            summary.append(step_summary)
        text = " ".join(summary)

        if verbosity == 2:
            print(text)
        
        if len(text) < step:
            return text
    return text

In [50]:
oversummarize(summarizer_bart, real_longest["review_text"], rounds=3)

Your max_length is set to 142, but you input_length is only 84. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 142, but you input_length is only 86. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)


' J.K. Rowling is a really excellent storyteller, and her voice reminded me of many of the old child characters . The story starts to feel like a blend of Matilda and Jane Eyre with a male protagonist . Harry Potter lives in a closet under the staircase, but simply because the Dursleys feel like putting him there .  The main kid players in the story all get introduced on the train . To my surprise, this was Hermione Granger in her earliest stage . Harry Potter\'s constant feelings of "I\'m-not-a-hero-you-must-be-wrong" probably informed Reynie Muldoon in The Mysterious Benedict Society .  Neville\'s not considered a coward: at the end, he\'s, quite rightly, rewarded for having the courage to stand up to friends . So why is all the merchandise about Harry, Ron and Hermione? I\'m also curious to know a little of Neville\'s backstory; why is he always with Hermione?'

In [51]:
def chunker(text, step):
    sents = nltk.sent_tokenize(text)
    tokenized = [nltk.word_tokenize(sent) for sent in sents]
    
    detokenizer = nltk.tokenize.treebank.TreebankWordDetokenizer()
    
    chunks = []
    chunk = []
    for sent in tokenized:
        if len(chunk) + len(sent) < step:
            chunk.extend(sent)
        else:
            chunks.append(detokenizer.detokenize(chunk))
            chunk = []
    
    if len(chunk) > 5:
        chunks.append(detokenizer.detokenize(chunk))

    return chunks

def oversummarize_sensitive(summarizer, text, rounds=3, step=512, verbosity=0):
    for r in range(1, rounds + 1):
        if verbosity > 0:
            print(f"ROUND {r}")
        
        chunks = chunker(text, step)
        summary = [step_sum["summary_text"] for step_sum in summarizer(chunks)]
        
        assert len(chunks) == len(summary)
        
        if verbosity > 1:
            for i in range(len(chunks)):
                print(chunks[i])
                print(summary[i])
                print()
        text = " ".join(summary)

        if verbosity == 1:
            print(text)

        if len(nltk.word_tokenize(text)) < step:
            return text
    return text

In [52]:
oversummarize_sensitive(summarizer_bart, real_longest["review_text"], step=500)

" J.K. Rowling is a really excellent storyteller . Her voice reminded me of many of the old children's authors I read when I was younger . Harry Potter's resonance is based on its roots in lesser-read classics that were, after all, classics for a reason . The Weasleys get some of the best dialogue in the book, particularly Fred and George, the twins ."

In [53]:
oversummarize(summarizer_pegasus, real_longest["review_text"])

Your max_length is set to 64, but you input_length is only 20. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


"In the latest Harry Potter spin-off, Harry Potter and the Cursed Child, we get to meet Hagrid, Harry's former teacher at Hogwarts. Harry Potter and the Philosopher's Stone is out on DVD and Blu-ray this week, and it's a pretty good introduction to the world of Harry Potter. The first trailer for the new Harry Potter spin-off, Fantastic Beasts and Where to Find Them, has been released."

In [54]:
oversummarize_sensitive(summarizer_pegasus, real_longest["review_text"], step=450)

"Harry Potter and the Cursed Child is the seventh and final book in the Harry Potter series. If you've read the Harry Potter books, you'll know that the Dursleys are not the nicest people in the world. In Harry Potter and the Philosopher's Stone, Harry is babysitting his aunt and uncle's four-year-old nephew. Harry Potter and the Philosopher's Stone begins with the Dursleys, Harry's family, getting their hands on all the letters that Voldemort sent to Harry's parents. Harry Potter and the Philosopher's Stone is the seventh and final book in the Harry Potter series. Harry and Ron go to Hogwarts for the first time in the seventh and final book in the Harry Potter series. In Harry Potter and the Philosopher's Stone, Harry and Hermione are determined to stop Snape from stealing the Sorcerer's Stone. I've been thinking a lot about Harry Potter and the Half-Blood Prince lately, and one of the things that struck me most was how much of the story is about Harry Potter's relationship with his H